# Workshop 1 - Implementation
    
In this notebook you will implement the matrix method and check it with some sanity checks.

Our matrix method implementation is now completely stored in a local package, consisting of three classes. If you need a refresher on how to code with Classes and Objects, refer to the [section on Object Oriented Programming in the MUDE-book](https://mude.citg.tudelft.nl/2024/book/external/learn-programming/book/python/oop/classes.html), with additionally [programming assignment 1.7](https://mude.citg.tudelft.nl/2024/files/Week_1_7/PA_1_7_classy_distributions.html).

In [279]:
import numpy as np
import matrixmethod as mm
%config InlineBackend.figure_formats = ['svg']

## 1. The Node class
This class is stored in `./matrixmethod/node.py`

The purpose of this class is to store node information and keep track of the total number of DOFs of the problem. Note the automatic bookkeeping we introduce in `__init__`. This simple but efficient way of keeping track of which DOFs belong to which nodes will make life much easier when we need to assemble matrices from multiple elements. The Node class doesn't need any modification.

> ### Task 1.1
> 
> To test whether you understand how the class works, create two nodes on coordinates ($0$,$0$) and ($3$,$4$) and print the string representation of both nodes. The `clear` function is called to restart the node and DOF counters. Make sure this is done whenever you start solving a new problem.


In [280]:
mm.Node.clear()

node1 = mm.Node(0, 0)
node2 = mm.Node(3, 4)
    
print(node1)
print(node2)

This node has:
 - x coordinate=0,
 - z coordinate=0,
 - degrees of freedom=[0, 1, 2],
 - load vector=[0. 0. 0.]
This node has:
 - x coordinate=3,
 - z coordinate=4,
 - degrees of freedom=[3, 4, 5],
 - load vector=[0. 0. 0.]


> Your output should look like this:
> 
> ```
> This node has:
>  - x coordinate=0,
>  - z coordinate=0,
>  - degrees of freedom=[0, 1, 2],
>  - load vector=[0. 0. 0.])
> This node has:
>  - x coordinate=3,
>  - z coordinate=4,
>  - degrees of freedom=[3, 4, 5],
>  - load vector=[0. 0. 0.])
> ```

## 2. The Element class
This class is stored in `./matrixmethod/elements.py`

This class keeps track of each element in the model, including:
- Cross-section properties
- Element orientation (for coordinate system transformations)
- Which Nodes make up each element, and in turn (with help of the Node class) which DOFs belong to each element

Apart from bookkeeping element data, the other main task of this class is to provide the element stiffness matrix in the global coordinate system (for subsequent assembly) and postprocess element-level fields. For now we keep postprocessing for next week and focus only on getting the correct stiffness matrix.

Here the class describes an element combining extension and Euler-Bernoulli bending. A similar (or inherited) class could also be implemented for different element types (*e.g.* shear beam, Timoshenko beam, cable elements, etc). Here we also keep it simple by assuming elements are all arranged in a 2D plane.

However, the implementation is incomplete:
- The transformation matrix is missing in `__init__`, which is given in the [online book](https://ciem5000-2025.github.io/book/lecture1/transformations.html). Make sure you take into account that a positive $\Delta z$ with a positive $\Delta x$ gives a negative angle $\alpha$. Make use of `numpy.arctan2` to return the angle between $-\pi$ and $\pi$, `numpy.arctan` returns an angle between $-\cfrac{\pi}{2}$ and $-\cfrac{\pi}{2}$, and therefore cannot distinguish between all four quadrants.
- The correct stiffness matrix for this extension-bending element coordinate system is missing in `stiffness`. You can derive the stiffness matrix yourself using pen and paper, SymPy or Maple, or copy the given stiffness matrix from the [online book](https://ciem5000-2025.github.io/book/lecture1/other_elements.html).
- We keep the functions which add a distributed load and compute the moments / displacements untouched for this week. Next week we'll implement those as well.


> ### Task 2.1
>
> Add the missing pieces to the code in `./matrixmethod/elements.py`, before you perform the checks below. Do you specify your stiffness matrix in the global or local coordinate system?

Whenever you make changes to your code in the `./matrixmethod/` folder, you need to reimport those. Instead of restarting the kernel, we use some magic ipython commands. Run the cell below once. Consequently, whenever you save your changes in one of the `.py`-files, it's automatically reloaded.

In [281]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


> ### Task 2.2
> 
>First, let's check the stiffness matrix for a beam which doesn't require rotation. Create a horizontal element with length $2$ and $EI=4$ and print both the transformation matrix and the stiffness matrix.
>
> Do the matrices match with what you'd expect?

In [282]:
mm.Node.clear()
mm.Element.clear()

In [283]:
node1 = mm.Node(0, 0)
node2 = mm.Node(2, 0)

elem = mm.Element(node1, node2)

section = {}
section['EI'] = 4  # Set the flexural stiffness
elem.set_section(section)

print("Transformation Matrix:")
print(elem.T)

print("\nStiffness Matrix:")
print(elem.stiffness())

Transformation Matrix:
[[ 1.  0.  0.  0.  0.  0.]
 [-0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0. -0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.]]

Stiffness Matrix:
[[ 5.e+19  0.e+00  0.e+00 -5.e+19  0.e+00  0.e+00]
 [ 0.e+00  6.e+00  6.e+00  0.e+00  0.e+00  0.e+00]
 [ 0.e+00  6.e+00  8.e+00  0.e+00  0.e+00  0.e+00]
 [-5.e+19  0.e+00  0.e+00  5.e+19  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00  6.e+00 -6.e+00]
 [ 0.e+00  0.e+00  0.e+00  0.e+00 -6.e+00  8.e+00]]


> ### Task 2.3
> Now, create a vertical element with length $2$ and $EI=4$ and print the transformation and stiffness matrix.
> 
> Do the matrices match with what you'd expect?

In [284]:
mm.Node.clear()
mm.Element.clear()

node1 = mm.Node(0, 0)
node2 = mm.Node(0, 2)

elem = mm.Element(node1, node2)

section = {}
section['EI'] = 4  # Set the flexural stiffness
elem.set_section(section)

print("Transformation Matrix:")
print(elem.T)

print("\nStiffness Matrix:")
print(elem.stiffness())

Transformation Matrix:
[[ 6.123234e-17  1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00]
 [-1.000000e+00  6.123234e-17  0.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00
   0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  6.123234e-17  1.000000e+00
   0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00 -1.000000e+00  6.123234e-17
   0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00
   1.000000e+00]]

Stiffness Matrix:
[[ 6.00000000e+00  3.06161700e+03 -6.00000000e+00 -1.87469973e-13
  -3.06161700e+03  0.00000000e+00]
 [ 3.06161700e+03  5.00000000e+19  3.67394040e-16 -3.06161700e+03
  -5.00000000e+19  0.00000000e+00]
 [-6.00000000e+00  3.67394040e-16  8.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.87469973e-13 -3.06161700e+03  0.00000000e+00  6.00000000e+00
   3.06161700e+03  6.00000000e+00]
 [-3.06161700e+03 -

> ### Task 2.4
>
>Now, create an element rotated $120 \degree$ with length $2$ and print the transformation matrix.
>
>Does it match with what you'd expect?

In [285]:
import numpy as np
import matrixmethod as mm

mm.Node.clear()
mm.Element.clear()

node1 = mm.Node(0, 0)
node2 = mm.Node(2, 0)

elem = mm.Element(node1, node2)

section = {}
section['EI'] = 4  # Set the flexural stiffness
elem.set_section(section)

alpha = 2 * np.pi / 3  # 120 degrees in radians

T = np.zeros((6, 6))

# For 120° rotation, the transformation matrix for a 2D beam element
T[0, 0] = T[1, 1] = T[3, 3] = T[4, 4] = np.cos(alpha)
T[0, 1] = T[3, 4] = np.sin(alpha)
T[1, 0] = T[4, 3] = - np.sin(alpha)
T[2, 2] = T[5, 5] = 1

elem.T = T
elem.Tt = np.transpose(T)

print("Transformation Matrix for 120° Rotation:")
print(elem.T)


Transformation Matrix for 120° Rotation:
[[-0.5        0.8660254  0.         0.         0.         0.       ]
 [-0.8660254 -0.5        0.         0.         0.         0.       ]
 [ 0.         0.         1.         0.         0.         0.       ]
 [ 0.         0.         0.        -0.5        0.8660254  0.       ]
 [ 0.         0.         0.        -0.8660254 -0.5        0.       ]
 [ 0.         0.         0.         0.         0.         1.       ]]


OR

In [286]:
mm.Node.clear()
mm.Element.clear()

node1 = mm.Node(0,0)
node2 = mm.Node(-1,-np.sqrt(3))
elem = mm.Element ( node1, node2 )

print(elem)
print(elem.T)

Element connecting:
node #1:
 This node has:
 - x coordinate=0,
 - z coordinate=0,
 - degrees of freedom=[0, 1, 2],
 - load vector=[0. 0. 0.]
with node #2:
 This node has:
 - x coordinate=-1,
 - z coordinate=-1.7320508075688772,
 - degrees of freedom=[3, 4, 5],
 - load vector=[0. 0. 0.]
[[-0.5       -0.8660254  0.         0.         0.         0.       ]
 [ 0.8660254 -0.5        0.         0.         0.         0.       ]
 [ 0.         0.         1.         0.         0.         0.       ]
 [ 0.         0.         0.        -0.5       -0.8660254  0.       ]
 [ 0.         0.         0.         0.8660254 -0.5        0.       ]
 [ 0.         0.         0.         0.         0.         1.       ]]


> ### Task 2.5
>
>Now, create an element rotated $60 \degree$ with length $2$ and print the transformation matrix.
>
>Does it match with what you'd expect?

In [287]:
mm.Node.clear()
mm.Element.clear()

node1 = mm.Node(0, 0)
node2 = mm.Node(2, 0)

elem = mm.Element(node1, node2)

section = {}
section['EI'] = 4  # Set the flexural stiffness
elem.set_section(section)

alpha = np.pi / 3  

T = np.zeros((6, 6))

T[0, 0] = T[1, 1] = T[3, 3] = T[4, 4] = np.cos(alpha)
T[0, 1] = T[3, 4] = np.sin(alpha)
T[1, 0] = T[4, 3] = - np.sin(alpha)
T[2, 2] = T[5, 5] = 1 

elem.T = T
elem.Tt = np.transpose(T)

print("Transformation Matrix for 120° Rotation:")
print(elem.T)


Transformation Matrix for 120° Rotation:
[[ 0.5        0.8660254  0.         0.         0.         0.       ]
 [-0.8660254  0.5        0.         0.         0.         0.       ]
 [ 0.         0.         1.         0.         0.         0.       ]
 [ 0.         0.         0.         0.5        0.8660254  0.       ]
 [ 0.         0.         0.        -0.8660254  0.5        0.       ]
 [ 0.         0.         0.         0.         0.         1.       ]]


OR

In [288]:
mm.Node.clear()
mm.Element.clear()

node1 = mm.Node(0,0)
node2 = mm.Node(1,-np.sqrt(3))
elem = mm.Element ( node1, node2 )

print(elem)
print(elem.T)

Element connecting:
node #1:
 This node has:
 - x coordinate=0,
 - z coordinate=0,
 - degrees of freedom=[0, 1, 2],
 - load vector=[0. 0. 0.]
with node #2:
 This node has:
 - x coordinate=1,
 - z coordinate=-1.7320508075688772,
 - degrees of freedom=[3, 4, 5],
 - load vector=[0. 0. 0.]
[[ 0.5       -0.8660254  0.         0.         0.         0.       ]
 [ 0.8660254  0.5        0.         0.         0.         0.       ]
 [ 0.         0.         1.         0.         0.         0.       ]
 [ 0.         0.         0.         0.5       -0.8660254  0.       ]
 [ 0.         0.         0.         0.8660254  0.5        0.       ]
 [ 0.         0.         0.         0.         0.         1.       ]]


> ### Task 2.6
> 
> For the previous element, a global displacement vector $\mathbf{u}^{(e)} = \begin{bmatrix} 0 \\0 \\ 0 \\ \sqrt{3} \\ 1 \\ 0 \end{bmatrix}$ is given. What would be displacement vector $\mathbf{u}^{(e)}$?
>
> Check your answer using pen and paper. Tip: make a drawing instead of doing all the algebra.


In [289]:
import numpy as np
import matrixmethod as mm

mm.Node.clear()
mm.Element.clear()

node1 = mm.Node(0, 0)
node2 = mm.Node(2, 0)

elem = mm.Element(node1, node2)

section = {}
section['EI'] = 4  # Set the flexural stiffness
elem.set_section(section)

alpha = np.pi / 3  # 60 degrees in radians

T = np.zeros((6, 6))

T[0, 0] = T[1, 1] = T[3, 3] = T[4, 4] = np.cos(alpha)
T[0, 1] = T[3, 4] = np.sin(alpha)
T[1, 0] = T[4, 3] = - np.sin(alpha)
T[2, 2] = T[5, 5] = 1 

elem.T = T
elem.Tt = np.transpose(T)

u_global = np.array([0, 0, 0, np.sqrt(3), 1, 0])

u_local = np.dot(np.transpose(T), u_global)

print("Local Displacement Vector (u^(l)):")
print(u_local)


Local Displacement Vector (u^(l)):
[0.00000000e+00 0.00000000e+00 0.00000000e+00 2.22044605e-16
 2.00000000e+00 0.00000000e+00]


OR

In [290]:
print(np.matmul(elem.T,np.array([0,0,0,np.sqrt(3),1,0])))

[ 0.          0.          0.          1.73205081 -1.          0.        ]


## 3. The Constrainer class
This class is stored in `./matrixmethod/constrainer.py`

This small class keeps track of which DOFs have prescribed displacements and takes care of applying these constraints to the global $\mathbf{K}$ and $\mathbf{f}$. For now we keep it simple and assume all constraints fix the DOF values to zero. Next week we will deal with non-zero prescribed values. 

However, the implementation is incomplete:
- The `constrain` function is incomplete, which should mimic the process of striking rows/columns of constrained DOFs and reduce the size of the system to be solved. Remember that `Constrainer` stores which DOFs are constrained in `self.dofs`, so **all the others** should be free. After gathering the free DOFs in an array, you will need to select the correct blocks of $\mathbf{K}$ and $\mathbf{f}$. For the stiffness matrix you will need the `np.ix_()` helper function (check its documentation [here](https://numpy.org/doc/stable/reference/generated/numpy.ix_.html))
- We keep the function which calculates supports reaction untouched for this week. Next week we'll implement that one as well.

> ### Task 3.1
> Add the missing pieces to the code, before you perform the check below

> ### Task 3.2
> 
> Take the inclined element of task 2.5 and a bending stiffness of $1$. What happens if you invert $\mathbf{K}$? Now fix all degrees of freedom of the first node. What happens when you invert your 'constrained' $\mathbf{K}$? Are the dimensions of the 'constrained' $\mathbf{K}$ correct?

In [291]:

mm.Node.clear()
mm.Element.clear()

node1 = mm.Node(0, 0)
node2 = mm.Node(2, 0)

elem = mm.Element(node1, node2)

section = {}
section['EI'] = 1  
elem.set_section(section)

alpha = np.pi / 3  

T = np.zeros((6, 6))

T[0, 0] = T[1, 1] = T[3, 3] = T[4, 4] = np.cos(alpha)
T[0, 1] = T[3, 4] = np.sin(alpha)
T[1, 0] = T[4, 3] = - np.sin(alpha)
T[2, 2] = T[5, 5] = 1 

elem.T = T
elem.Tt = np.transpose(T)

K = elem.stiffness()

print("stiffness matrix (K):")
print(K)

try:
    K_inv = np.linalg.inv(K)
    print("\nInverse of the Stiffness Matrix (K_inv):")
    print(K_inv)
except np.linalg.LinAlgError:
    print("\nError: The stiffness matrix is singular and cannot be inverted.")

con = mm.Constrainer()
con.fix_node(node1)
f = np.zeros(6)

Kff, Fff = con.constrain(k=K, f=f)

print("\nConstrained Stiffness Matrix (Kff):")
print(Kff)

print("\nShape of the Reduced Stiffness Matrix (K_ff):", np.shape(Kff))


stiffness matrix (K):
[[ 1.25000000e+19  2.16506351e+19 -1.29903811e+00 -1.25000000e+19
  -2.16506351e+19  0.00000000e+00]
 [ 2.16506351e+19  3.75000000e+19  7.50000000e-01 -2.16506351e+19
  -3.75000000e+19  0.00000000e+00]
 [-1.29903811e+00  7.50000000e-01  2.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.25000000e+19 -2.16506351e+19  0.00000000e+00  1.25000000e+19
   2.16506351e+19  1.29903811e+00]
 [-2.16506351e+19 -3.75000000e+19  0.00000000e+00  2.16506351e+19
   3.75000000e+19 -7.50000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.29903811e+00
  -7.50000000e-01  2.00000000e+00]]

Inverse of the Stiffness Matrix (K_inv):
[[ 6.80125019e+15  1.17801109e+16 -4.63210897e-01  6.80125019e+15
   1.17801109e+16 -7.53190508e-03]
 [ 1.74807698e+16  3.02775815e+16 -5.23537933e-01  1.74807698e+16
   3.02775815e+16 -6.85671948e-01]
 [-2.13774711e+15 -3.70268661e+15  3.95462422e-01 -2.13774711e+15
  -3.70268661e+15  2.52234865e-01]
 [ 1.17381657e+16  2.03310

WHAT THEY WANTED

In [292]:
section = {}
section['EI'] = 1
elem.set_section (section)

k = elem.stiffness()
#print(np.linalg.inv(k))

con = mm.Constrainer()

con.fix_node (node1)

print(con)

f = np.zeros (6) #empty load vector
Kff, Fff = con.constrain( k, f )
print(np.shape(np.linalg.inv(Kff)))

This constrainer has constrained the degrees of freedom: [0, 1, 2] with corresponding constrained values: [0, 0, 0])
(3, 3)


## 4. Full implementation extension bar

Having made our implementations, we now check them with two simple examples that serve as sanity checks. The first is a simple bar undergoing extension:

<center><figure>
  <IMG SRC="https://raw.githubusercontent.com/ibcmrocha/public/main/extpointload.png" WIDTH=200 ALIGN="center">
      </figure></center>

With $EA = 1000$, $F = 100$ and $L = 1$.

Use the code blocks below to set up and solve this problem using the classes above. The steps to follow are outlined below and short explanations/hints are given. Once you have a solution for the horizontal displacement of the node at the right end of the bar, compare it to the analytical solution you obtained in the first half of the course.

In [293]:
mm.Node.clear()
mm.Element.clear()

> ### Task 4.1: 
> 
> Create two nodes here. You can store them on a `list` or simply create them as two separate objects (*e.g.* `node1` and `node2`). 

In [294]:
node1 = mm.Node(0, 0)  # Node 1 at the left end (x=0)
node2 = mm.Node(1, 0)  # Node 2 at the right end (x=1)

nodes = [node1, node2]

print("Node 1:", node1)
print("Node 2:", node2)


Node 1: This node has:
 - x coordinate=0,
 - z coordinate=0,
 - degrees of freedom=[0, 1, 2],
 - load vector=[0. 0. 0.]
Node 2: This node has:
 - x coordinate=1,
 - z coordinate=0,
 - degrees of freedom=[3, 4, 5],
 - load vector=[0. 0. 0.]


> ### Task 4.2
> 
> Here we only have a single element, so there is no need to store it in a `list` yet. You are also going to need a `dict` defining the cross-section of the element.

In [295]:
elem = mm.Element(node1, node2)

section = {}
section['EA'] = 1000 
elem.set_section(section)

print("Element connecting Node 1 and Node 2:")
print(elem)
print("\nSection properties (EA):")
print(section)


Element connecting Node 1 and Node 2:
Element connecting:
node #1:
 This node has:
 - x coordinate=0,
 - z coordinate=0,
 - degrees of freedom=[0, 1, 2],
 - load vector=[0. 0. 0.]
with node #2:
 This node has:
 - x coordinate=1,
 - z coordinate=0,
 - degrees of freedom=[3, 4, 5],
 - load vector=[0. 0. 0.]

Section properties (EA):
{'EA': 1000}


> ### Task 4.3
> 
> Let's define the boundary conditions. We create an instance of the `Constrainer` class to deal with prescribed displacements. Take a look at its functions and inform if Node 1 is fully fixed.
> You also need to pass the load $F$ on to Node 2. Check the member functions of `Node` to infer how that should be done.

In [296]:
con = mm.Constrainer()
con.fix_node(node1)

F = 100
node2.add_load(np.array([F, 0, 0]))  

print("Node 1 load vector:")
print(node1.p)

print("Node 2 load vector:")
print(node2.p)

print("\nNode 1's degrees of freedom (fixed):")
print(node1.dofs)



Node 1 load vector:
[0. 0. 0.]
Node 2 load vector:
[100.   0.   0.]

Node 1's degrees of freedom (fixed):
[0, 1, 2]


> ### Task 4.4
> 
> Now assemble the global stiffness matrix and force vector. Since we only have one element, there is no real assembly to be performed other than getting the stiffness matrix of the single element and storing the load at Node 2 in the correct positions of $\mathbf{f}$.

In [297]:
K = elem.stiffness()

f_global = np.zeros(6)

f_global[3] = F 

print("Global Stiffness Matrix (K):")
print(K)

print("\nGlobal Force Vector (f):")
print(f_global)


Global Stiffness Matrix (K):
[[ 1.0e+03  0.0e+00  0.0e+00 -1.0e+03  0.0e+00  0.0e+00]
 [ 0.0e+00  1.2e+21  6.0e+20  0.0e+00  0.0e+00  0.0e+00]
 [ 0.0e+00  6.0e+20  4.0e+20  0.0e+00  0.0e+00  0.0e+00]
 [-1.0e+03  0.0e+00  0.0e+00  1.0e+03  0.0e+00  0.0e+00]
 [ 0.0e+00  0.0e+00  0.0e+00  0.0e+00  1.2e+21 -6.0e+20]
 [ 0.0e+00  0.0e+00  0.0e+00  0.0e+00 -6.0e+20  4.0e+20]]

Global Force Vector (f):
[  0.   0.   0. 100.   0.   0.]


ACTUAL

In [298]:
global_k = elem.stiffness()
global_f = np.zeros (6)

global_f[3:6] = node2.p

> ### Task 4.5
> 
> Constrain the problem and solve for nodal displacements

In [299]:
Kff, Fff = con.constrain(K, f_global)

u_free = np.linalg.solve(Kff, Fff)

u_full = con.full_disp(u_free)

print("\nReduced Stiffness Matrix (Kff):")
print(Kff)

print("\nReduced Force Vector (Fff):")
print(Fff)

print("\nDisplacements of Free DOFs (u_free):")
print(u_free)

print("\nFull Displacement Vector (u_full):")
print(u_full)



Reduced Stiffness Matrix (Kff):
[[ 1.0e+03  0.0e+00  0.0e+00]
 [ 0.0e+00  1.2e+21 -6.0e+20]
 [ 0.0e+00 -6.0e+20  4.0e+20]]

Reduced Force Vector (Fff):
[100.   0.   0.]

Displacements of Free DOFs (u_free):
[0.1 0.  0. ]

Full Displacement Vector (u_full):
[0.  0.  0.  0.1 0.  0. ]


ACTUAL

In [300]:
Kff, Ff = con.constrain ( global_k, global_f )
u = np.matmul ( np.linalg.inv(Kff), Ff )
print(u)

[0.1 0.  0. ]


> ### Task 4.6
> 
> Finally, compare the displacement at the end of the bar with the one coming from the ODE solution. Note that since our element is already suitable for frames combining extension and bending, $\mathbf{u}$ has three entries. Which one is the entry that matters to us here? Did your solutions match? If so, that is a sign your implementation is correct. Can you use the function `full_disp` to obtain a vector of all displacements?

In [301]:
u_free = np.linalg.solve(Kff, Fff)

print("\nDisplacements of Free DOFs (u_free):")
print(u_free)

u_full = con.full_disp(u_free)

print("\nFull Displacement Vector (u_full):")
print(u_full)

displacement_at_end = u_full[3]

analytical_solution = 100 * 1 / 1000  # F * L / EA
print("\nDisplacement at the end of the bar (Node 2) from the solution:", displacement_at_end)
print("Analytical solution for displacement at Node 2:", analytical_solution)

if np.isclose(displacement_at_end, analytical_solution):
    print("\nThe displacement matches the analytical solution! The implementation is correct.")
else:
    print("\nThe displacement does not match the analytical solution. There might be an issue with the implementation.")



Displacements of Free DOFs (u_free):
[0.1 0.  0. ]

Full Displacement Vector (u_full):
[0.  0.  0.  0.1 0.  0. ]

Displacement at the end of the bar (Node 2) from the solution: 0.1
Analytical solution for displacement at Node 2: 0.1

The displacement matches the analytical solution! The implementation is correct.


ACTUAL

In [302]:
con.full_disp(u)

array([0. , 0. , 0. , 0.1, 0. , 0. ])

## Task 5: Full implementation bending beam

In the first example above we tested our model under extension. But that does not really guarantee it will behave correctly in bending. That is the goal of this second sanity check. Let's solve the following problem:

<center><figure>
  <IMG SRC="https://raw.githubusercontent.com/ibcmrocha/public/main/cantilever.png" WIDTH=200 ALIGN="center">
      </figure></center>

Choose appropriate values yourself

When setting up and solving your model, note that we are now interested in $w$ displacements, our load is now vertical and the cross-section property driving our deformation is now $EI$. Good luck!

In [303]:
mm.Node.clear()
mm.Element.clear()

> ### Task 5.1: Create nodes

In [304]:
node1 = mm.Node(0, 0)
node2 = mm.Node(2, 0)

nodes = [node1, node2]

print("Node 1:", node1)
print("Node 2:", node2)


Node 1: This node has:
 - x coordinate=0,
 - z coordinate=0,
 - degrees of freedom=[0, 1, 2],
 - load vector=[0. 0. 0.]
Node 2: This node has:
 - x coordinate=2,
 - z coordinate=0,
 - degrees of freedom=[3, 4, 5],
 - load vector=[0. 0. 0.]


> ### Task 5.2: Create element

In [305]:
elem = mm.Element(node1, node2)

section = {}
section['EI'] = 4000
elem.set_section(section)

print("Element connecting Node 1 and Node 2:")
print(elem)
print("\nSection properties (EI):")
print(section)


Element connecting Node 1 and Node 2:
Element connecting:
node #1:
 This node has:
 - x coordinate=0,
 - z coordinate=0,
 - degrees of freedom=[0, 1, 2],
 - load vector=[0. 0. 0.]
with node #2:
 This node has:
 - x coordinate=2,
 - z coordinate=0,
 - degrees of freedom=[3, 4, 5],
 - load vector=[0. 0. 0.]

Section properties (EI):
{'EI': 4000}


> ### Task 5.3: Set boundary conditions

In [306]:
con = mm.Constrainer()
con.fix_node(node1)
F = 100
node2.add_load([0, F, 0])
print(con)

This constrainer has constrained the degrees of freedom: [0, 1, 2] with corresponding constrained values: [0, 0, 0])


> ### Task 5.4: Assemble the system of equations

In [307]:
global_k = elem.stiffness()
global_f = np.zeros (6)

global_f[0:3] = node1.p
global_f[3:6] = node2.p

> ### Task 5.5: Constrain the problem and solve for nodal displacements

In [308]:
Kff, Fff = con.constrain(K, f_global)
u_free = np.linalg.solve(Kff, Fff)

print("\nDisplacements of Free DOFs (u_free):")
print(u_free)

u_full = con.full_disp(u_free)

print("\nFull Displacement Vector (u_full):")
print(u_full)

displacement_at_end = u_full[4]


Displacements of Free DOFs (u_free):
[0.1 0.  0. ]

Full Displacement Vector (u_full):
[0.  0.  0.  0.1 0.  0. ]


ACTUAL

In [309]:
Kff, Ff = con.constrain ( global_k, global_f )
u = np.matmul ( np.linalg.inv(Kff), Ff )
print(u)

[0.         0.06666667 0.05      ]


> ### Task 5.6: Check with the analytical solution
> 
> Did your solutions match? If so, your implementation is correct!

In [310]:
# Analytical solution for displacement due to bending
# Using the formula for vertical displacement of a cantilever beam:
# w = F * L^3 / (3 * EI)
analytical_solution = F * (2 ** 3) / (3 * 4000)

print("\nVertical Displacement at the end of the beam (Node 2) from the solution:", displacement_at_end)
print("Analytical solution for vertical displacement at Node 2:", analytical_solution)

if np.isclose(displacement_at_end, analytical_solution):
    print("\nThe displacement matches the analytical solution! The implementation is correct.")
else:
    print("\nThe displacement does not match the analytical solution. There might be an issue with the implementation.")


Vertical Displacement at the end of the beam (Node 2) from the solution: 0.0
Analytical solution for vertical displacement at Node 2: 0.06666666666666667

The displacement does not match the analytical solution. There might be an issue with the implementation.
